In [1]:
import os
import pickle
import glob
from tqdm import tqdm
from SkeletonExtraction.HRNET import *

D:\Anaconda\Main\envs\HRNet\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
C:\Users\ACER\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco-person/faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth


In [2]:
video_paths = "../data/nipun_video_dataset/PAMAP2_K10_V1"
save_paths = "HRNetSkeletons"
video_set_name = "nipun_video_dataset/PAMAP2_K10_V1"

In [3]:
vid_list = gen_video_list(video_paths)

In [5]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=3) as executor:
    skeletons = list(tqdm(executor.map(HRNetExtraction,vid_list), total=len(vid_list),desc="Processed Files:"))

Processed Files:: 100%|██████████| 177/177 [6:39:15<00:00, 135.34s/it]  


In [6]:
os.makedirs(os.path.join(save_paths,video_set_name),exist_ok=True)
with open(os.path.join(save_paths,video_set_name,f'combined.pkl'),"wb") as f0:
    pickle.dump(skeletons,f0)

In [7]:
with open(os.path.join(save_paths,video_set_name,f'combined.pkl'),"rb") as f0:
    skeletons = pickle.load(f0)

In [8]:
connections = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]

In [9]:
for skeleton in skeletons:
    print(skeleton.keys())
    break

dict_keys(['frame_dir', 'file_path', 'filename', 'label', 'img_shape', 'original_shape', 'total_frames', 'num_person_raw', 'keypoint', 'keypoint_score'])


In [10]:
set_extracted = set()
set_original = set()

for path in glob.glob("../data/nipun_video_dataset/PAMAP2_K10_V1/*/*.mp4"):
    set_original.add(path.split(os.path.sep)[-1][:-4])

for skeleton in skeletons:
    set_extracted.add(skeleton["frame_dir"])


set_extracted - set_original

set()

In [11]:
import cv2

os.makedirs(os.path.join(save_paths,video_set_name,"videos"),exist_ok=True)
for skeleton in tqdm(skeletons):
    width = skeleton["original_shape"][0]
    height = skeleton["original_shape"][1]

    size = (width, height)
    result = cv2.VideoWriter(os.path.join(save_paths,video_set_name,"videos",f"{skeleton['frame_dir']}.mp4"),
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)

    frames = extract_frame(skeleton['file_path'])
    #print(skeleton["keypoint"].astype(np.uint8))
    for f_no,frame in enumerate(frames):
        for body in skeleton["keypoint"]:
            #print(body.shape,frame.shape)
            for i, coord in enumerate(body[f_no]):
                x, y = int(coord[0]),int(coord[1])
                #print(x,y)
                frame = cv2.circle(frame, center=(x, y), radius=3, color=(255, 0, 0), thickness=6)

            for line in connections:
                i, j = line
                st = body[f_no][i, :]
                start_point = (int(width * st[0]), int(height * st[1]))

                en = body[f_no][j, :]
                end_point = (int(width * en[0]), int(height * en[1]))

                frame = cv2.line(frame, start_point, end_point, color=(0, 0, 255), thickness=3)

        #result.write(frame)
        os.makedirs(os.path.join(save_paths,video_set_name,"videos",f"{skeleton['frame_dir']}"),exist_ok=True)
        cv2.imwrite(os.path.join(save_paths,video_set_name,"videos",f"{skeleton['frame_dir']}",f"{f_no}.png"),frame)


KeyboardInterrupt

